In [5]:
import requests
import json
import urllib.request
import pandas as pd
import ast

In [6]:
response_details_merge = pd.read_csv('../data/Data 6 - Merged Responses and Movie Details.csv', encoding='unicode_escape')
response_details_merge.head()

,Unnamed: 0,id_x,person_mbti,mbti_test,character,movie,new_title,genre_ids,id_y,popularity,release_date,title,vote_average,vote_count
0,0,0,ENFP,michaelcaloz.com,Tony Stark,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,24/04/2019,Avengers: Endgame,8.255,24867
1,1,0,ENFP,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,03/05/2023,Guardians of the Galaxy Vol. 3,7.972,6262
2,2,0,ENFP,michaelcaloz.com,Captain America,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,24/04/2019,Avengers: Endgame,8.255,24867
3,3,0,ENFP,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,28/06/2019,Spider-Man: Far From Home,7.400,15144
4,4,0,ENFP,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,31/05/2023,Spider-Man: Across the Spider-Verse,8.366,6265


# Actions
### Columns to be removed
- `Unnamed: 0`
- `movie` and `title` since `new_title` was assigned to hold the actual movie title in Script 4
- `vote_count`
### Renames
- `id_x` as `person_id`
- `new_title` as `movie`
- `id_y` as `movie_id`
- `popularity` as `movie_popularity`
- `vote_average` as `movie_vote_avg`
### Others
- Change `release_date` to only capture year information as `movie_year`


In [7]:
response_details = response_details_merge.copy()

response_details.drop(columns = ['Unnamed: 0', 'person_mbti', 'movie', 'title', 'vote_count'], inplace=True)
response_details.rename(columns = {'id_x':'person_id', 'new_title':'movie', 'id_y':'movie_id', 'popularity':'movie_popularity', 'vote_average':'movie_vote_avg'}, inplace = True)
response_details['movie_year'] = response_details_merge['release_date'].astype(str).str[:4]
response_details.drop(columns = ['release_date'], inplace=True)

response_details

,person_id,mbti_test,character,movie,genre_ids,movie_id,movie_popularity,movie_vote_avg,movie_year
0,0,michaelcaloz.com,Tony Stark,Avengers: Endgame,"[12, 878, 28]",299534,240.886,8.255,24/0
1,0,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,7.972,03/0
2,0,michaelcaloz.com,Captain America,Avengers: Endgame,"[12, 878, 28]",299534,240.886,8.255,24/0
3,0,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,7.400,28/0
4,0,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,8.366,31/0
...,...,...,...,...,...,...,...,...,...
700,123,michaelcaloz.com,Agent Smith,The Matrix,"[28, 878]",603,575.344,8.215,31/0
701,123,michaelcaloz.com,Daniel Plainview,There Will Be Blood,[18],7345,36.996,8.084,26/1
702,123,michaelcaloz.com,Eraserhead Baby,Eraserhead,"[14, 27]",985,67.775,7.400,03/0
703,123,michaelcaloz.com,Patrick Bateman,American Psycho,"[53, 18, 80]",1359,84.339,7.404,13/0


In [9]:
char_mbti_df = pd.read_csv('../data/Data 8 - Franchise-expanded Movies from Data 2.csv', encoding='unicode_escape')
char_mbti_df

,char,movie,mbti,votes
0,Guido Anselmi,8 1/2,INTP,57
1,King Leonidas of Sparta,300,ENTJ,460
2,Artemisia,300,ENTJ,127
3,Xerxes,300,INTJ,103
4,Gorgo,300,INFJ,62
...,...,...,...,...
10718,Gazelle,Zootopia (2016),ESFP,107
10719,Mr Manchas,Zootopia (2016),ISFJ,40
10720,Doug Ramses,Zootopia (2016),INTP,30
10721,Travis,Zootopia (2016),ESTP,27


In [ ]:
zero = response_details.iloc[-1]
char = zero['character']
movie = zero['movie']
print(char, movie)

The Duke Fear and Loathing in Las Vegas


In [ ]:
char_mbti_df_copy = char_mbti_df.copy()
query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)
# query_bool =  query_bool[query_bool.values==True]
char_mbti_df_copy = char_mbti_df_copy[query_bool]
char_mbti_df_copy
# print(query_bool)

,char,movie,mbti,votes


In [11]:
char_mbti_df_copy = char_mbti_df.copy()

failed_index = []
failed_char = []
failed_movie = []

k = len(response_details)
# k = 5

for i in range(k):
    zero = response_details.iloc[i]
    char = zero['character']
    movie = zero['movie']
    # print(char, movie)

    query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)
    char_mbti_query = char_mbti_df_copy[query_bool]
    try:
        char_df_subset = char_mbti_query.iloc[0]
        # print(i, char_df_subset['char'], char_df_subset['mbti'])
    except:
        print(i, '      FAILED!  ', char,  '    ', movie)
        failed_index.append(i)
        failed_char.append(char)
        failed_movie.append(movie)

failed_df = pd.DataFrame({'failed_index': failed_index, 'failed_char': failed_char, 'failed_movie': failed_movie})
failed_df

23       FAILED!   Mathilda      Mathilda
24       FAILED!   Willy Wonka      Willy Wonka & the Chocolate Factory
25       FAILED!   Remus Lupin      Harry Potter and the Philosopher's Stone
26       FAILED!   WALL-E      WALLÂ·E
27       FAILED!   Waymond Wang      Everything Everywhere All at Once
32       FAILED!   Grace Le domas      Ready or Not
33       FAILED!   Naomi Misora      Death Note Relight 1: Visions of a God
34       FAILED!   Yorichii       Demon Slayer: Kimetsu no Yaiba -To the Hashira Training-
39       FAILED!   Otto      A Man Called Otto
44       FAILED!   Jang Okju      Ballerina
45       FAILED!   Sienna Shaw      Terrifier 2
46       FAILED!   Jordan Turner      The Call
55       FAILED!   Jiro Horikoshi      The Wind Rises
56       FAILED!   Gretta      Begin Again
57       FAILED!   Sakura Yamauchi      Let Me Eat Your Pancreas
58       FAILED!   Makoto Konno      The Girl Who Leapt Through Time
59       FAILED!   Anthony Edward Stark       Iron Man 2
63    

C:\Users\Marco\AppData\Local\Temp\ipykernel_26800\4136469517.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)


215       FAILED!   Cruella de Vil       Cruella
217       FAILED!   Maximus      Gladiator
218       FAILED!   Howl      Howl's Moving Castle
221       FAILED!   Evelyn Salt      Saltburn
223       FAILED!   Suzume Iwato      Suzume
224       FAILED!   Professor Langdon      The Da Vinci Code
225       FAILED!   bowser      The Super Mario Bros. Movie
226       FAILED!   spider-punk      Spider-Man: Across the Spider-Verse
227       FAILED!   iron man      Avengers: Endgame
228       FAILED!   captain underpants      Captain Underpants: The First Epic Movie
232       FAILED!   Yas      Rye Lane
233       FAILED!   Alice      Plus One
236       FAILED!   Sasha Tran      Always Be My Maybe
244       FAILED!   Dr. Emmett Brown      Back to the Future
247       FAILED!   Shun      From Up on Poppy Hill
248       FAILED!   Park Ki Joon      Midnight Runners
249       FAILED!   Padre Burgos      GomBurZa
250       FAILED!   Howl      Howl's Moving Castle
252       FAILED!   Howl      Howl's

,failed_index,failed_char,failed_movie
0,23,Mathilda,Mathilda
1,24,Willy Wonka,Willy Wonka & the Chocolate Factory
2,25,Remus Lupin,Harry Potter and the Philosopher's Stone
3,26,WALL-E,WALLÂ·E
4,27,Waymond Wang,Everything Everywhere All at Once
...,...,...,...
306,696,Kiri,Avatar: The Way of Water
307,697,Terminator,Terminator 3: Rise of the Machines
308,699,Uta,One Piece Film Red
309,702,Eraserhead Baby,Eraserhead
